In [6]:
!pip install tensorflow optuna matplotlib scikit-learn

import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# Carregar o dataset CIFAR-10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalização dos dados de imagem
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Codificação one-hot para as etiquetas de classe
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Verificar as dimensões dos dados
x_train.shape, x_test.shape, y_train.shape, y_test.shape


((50000, 32, 32, 3), (10000, 32, 32, 3), (50000, 10), (10000, 10))

In [7]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# Carregar o dataset CIFAR-10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalização dos dados de imagem
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Codificação one-hot para as etiquetas de classe
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Verificar as dimensões dos dados
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)


(50000, 32, 32, 3) (10000, 32, 32, 3) (50000, 10) (10000, 10)


#  Definir a Estrutura da CNN

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def create_cnn_model(n_layers, n_filters, kernel_size, dropout_rate, optimizer):
    model = Sequential()
    model.add(Conv2D(n_filters, kernel_size, activation='relu', input_shape=(32, 32, 3)))
    
    for _ in range(n_layers - 1):
        model.add(Conv2D(n_filters, kernel_size, activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(dropout_rate))

    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(10, activation='softmax'))

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model


# Configuração do Optuna para Otimização de Hiperparâmetros

In [ ]:
import optuna
import os
import logging
import sys

# Definir uma função de callback para exibir informações de progresso
def print_progress(study, trial):
    print(f"Trial {trial.number} finished with value: {trial.value} and parameters: {trial.params}.")
    print(f"Best trial so far: Trial {study.best_trial.number} with value: {study.best_trial.value}.")

def objective(trial):
    n_layers = trial.suggest_int('n_layers', 1, 3)
    n_filters = trial.suggest_categorical('n_filters', [32, 64, 128])
    kernel_size = trial.suggest_categorical('kernel_size', [(3, 3), (5, 5)])
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
    optimizer = trial.suggest_categorical('optimizer', ['adam', 'sgd'])

    model = create_cnn_model(n_layers, n_filters, kernel_size, dropout_rate, optimizer)
    model.fit(x_train, y_train, validation_split=0.1, epochs=7, verbose=0)
    
    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
    return accuracy

# Configurar o logging
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
optuna.logging.set_verbosity(optuna.logging.INFO)

# Definir o número máximo de threads
max_threads = os.cpu_count()

# Criar e executar o estudo do Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=16, n_jobs=max_threads, callbacks=[print_progress])


[I 2023-12-16 19:00:12,536] A new study created in memory with name: no-name-a7657ce1-7c35-4e26-9f88-54424e609153


A new study created in memory with name: no-name-a7657ce1-7c35-4e26-9f88-54424e609153


C:\Users\ealbvit\AppData\Local\Temp\ipykernel_15316\3697548424.py:15: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
C:\Users\ealbvit\AppData\Local\anaconda3\envs\redes_neurais\lib\site-packages\optuna\distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (3, 3) which is of type tuple.
  warnings.warn(message)
C:\Users\ealbvit\AppData\Local\anaconda3\envs\redes_neurais\lib\site-packages\optuna\distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (5, 5) which is of type tuple.
  warnings.warn(message)
[I 2023-12-16 19:45:43,024] Trial 6 finished with value: 0.5749999880790

Trial 6 finished with value: 0.574999988079071 and parameters: {'n_layers': 1, 'n_filters': 64, 'kernel_size': (5, 5), 'dropout_rate': 0.27026550952370554, 'optimizer': 'sgd'}. Best is trial 6 with value: 0.574999988079071.
Trial 6 finished with value: 0.574999988079071 and parameters: {'n_layers': 1, 'n_filters': 64, 'kernel_size': (5, 5), 'dropout_rate': 0.27026550952370554, 'optimizer': 'sgd'}.
Best trial so far: Trial 6 with value: 0.574999988079071.


[I 2023-12-16 19:47:50,931] Trial 7 finished with value: 0.5450000166893005 and parameters: {'n_layers': 1, 'n_filters': 32, 'kernel_size': (5, 5), 'dropout_rate': 0.41639259237258797, 'optimizer': 'adam'}. Best is trial 6 with value: 0.574999988079071.


...

Trial 1 finished with value: 0.6470000147819519 and parameters: {'n_layers': 3, 'n_filters': 64, 'kernel_size': (3, 3), 'dropout_rate': 0.4844914550019569, 'optimizer': 'adam'}. Best is trial 5 with value: 0.6650000214576721.
Trial 1 finished with value: 0.6470000147819519 and parameters: {'n_layers': 3, 'n_filters': 64, 'kernel_size': (3, 3), 'dropout_rate': 0.4844914550019569, 'optimizer': 'adam'}.
Best trial so far: Trial 5 with value: 0.6650000214576721.


[I 2023-12-16 20:51:23,380] Trial 10 finished with value: 0.5820000171661377 and parameters: {'n_layers': 3, 'n_filters': 64, 'kernel_size': (3, 3), 'dropout_rate': 0.18368745069898018, 'optimizer': 'sgd'}. Best is trial 5 with value: 0.6650000214576721.


Trial 10 finished with value: 0.5820000171661377 and parameters: {'n_layers': 3, 'n_filters': 64, 'kernel_size': (3, 3), 'dropout_rate': 0.18368745069898018, 'optimizer': 'sgd'}. Best is trial 5 with value: 0.6650000214576721.
Trial 10 finished with value: 0.5820000171661377 and parameters: {'n_layers': 3, 'n_filters': 64, 'kernel_size': (3, 3), 'dropout_rate': 0.18368745069898018, 'optimizer': 'sgd'}.
Best trial so far: Trial 5 with value: 0.6650000214576721.


[I 2023-12-16 21:21:21,629] Trial 13 finished with value: 0.6545000076293945 and parameters: {'n_layers': 3, 'n_filters': 64, 'kernel_size': (5, 5), 'dropout_rate': 0.14530599921142928, 'optimizer': 'adam'}. Best is trial 5 with value: 0.6650000214576721.


Trial 13 finished with value: 0.6545000076293945 and parameters: {'n_layers': 3, 'n_filters': 64, 'kernel_size': (5, 5), 'dropout_rate': 0.14530599921142928, 'optimizer': 'adam'}. Best is trial 5 with value: 0.6650000214576721.
Trial 13 finished with value: 0.6545000076293945 and parameters: {'n_layers': 3, 'n_filters': 64, 'kernel_size': (5, 5), 'dropout_rate': 0.14530599921142928, 'optimizer': 'adam'}.
Best trial so far: Trial 5 with value: 0.6650000214576721.


# Análise dos Resultados do Optuna

In [ ]:
from optuna.visualization import plot_slice, plot_contour, plot_parallel_coordinate, plot_param_importances
from IPython.display import display

# Slice Plot
display(plot_slice(study))

# Contour Plot
display(plot_contour(study))

# Parallel Coordinate Plot
display(plot_parallel_coordinate(study))

# Importance Plot
display(plot_param_importances(study))


Ficou evidente que o numero de layers é o hiperparametro mais relevante. Como o maximo no meu estudo ainda nao parece ser o melhor, faremos agora novos estudos com os melhores hiperparametros obtidos e iremos variar apenas o n_layers

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import tensorflow as tf

# Função para calcular e plotar a matriz de confusão
def plot_confusion_matrix(model, x_test, y_test, title):
    y_pred = model.predict(x_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y_test, axis=1)
    conf_matrix = confusion_matrix(y_true, y_pred_classes)

    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.title(title)
    plt.show()


In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def train_model(params):
    model = create_cnn_model(**params)
    model.fit(x_train, y_train, epochs=30, verbose=1)
    return model

def train_and_plot_confusion_matrix(best_trial_params):
    print("Treinando modelo com hiperparâmetros:", best_trial_params)
    model = train_model(best_trial_params)
    title = 'Matriz de Confusão\nHiperparâmetros: ' + str(best_trial_params)
    plot_confusion_matrix(model, x_test, y_test, title)

best_trials = sorted(study.trials, key=lambda t: t.value, reverse=True)
best_trial_params_1 = best_trials[0].params
best_trial_params_2 = best_trials[1].params

with ThreadPoolExecutor(max_workers=2) as executor:
    futures = [executor.submit(train_and_plot_confusion_matrix, best_trial_params_1),
               executor.submit(train_and_plot_confusion_matrix, best_trial_params_2)]

    for future in as_completed(futures):
        future.result()  # Aguardar cada treinamento ser concluído


In [ ]:

from concurrent.futures import ThreadPoolExecutor, as_completed

# Definindo os melhores hiperparâmetros diretamente
best_trial_params_1 = {'n_layers': 2, 'n_filters': 32, 'kernel_size': (3, 3), 'dropout_rate': 0.3164845495404709, 'optimizer': 'adam'}  # Trial 5
best_trial_params_2 = {'n_layers': 1, 'n_filters': 64, 'kernel_size': (5, 5), 'dropout_rate': 0.27026550952370554, 'optimizer': 'sgd'}  # Trial 6

def train_model(params):
    model = create_cnn_model(**params)
    model.fit(x_train, y_train, epochs=30, verbose=1)
    return model

def train_and_plot_confusion_matrix(best_trial_params):
    print("Treinando modelo com hiperparâmetros:", best_trial_params)
    model = train_model(best_trial_params)
    title = 'Matriz de Confusão\nHiperparâmetros: ' + str(best_trial_params)
    plot_confusion_matrix(model, x_test, y_test, title)

with ThreadPoolExecutor(max_workers=2) as executor:
    futures = [executor.submit(train_and_plot_confusion_matrix, best_trial_params_1),
               executor.submit(train_and_plot_confusion_matrix, best_trial_params_2)]

    for future in as_completed(futures):
        future.result()  # Aguardar cada treinamento ser concluído


Treinando modelo com hiperparâmetros: {'n_layers': 2, 'n_filters': 32, 'kernel_size': (3, 3), 'dropout_rate': 0.3164845495404709, 'optimizer': 'adam'}
Treinando modelo com hiperparâmetros: {'n_layers': 1, 'n_filters': 64, 'kernel_size': (5, 5), 'dropout_rate': 0.27026550952370554, 'optimizer': 'sgd'}
Epoch 1/30
Epoch 1/30
1563/1563 [==============================] - 86s 54ms/step - loss: 1.9217 - accuracy: 0.3045
Epoch 2/30
1563/1563 [==============================] - 87s 56ms/step - loss: 1.5920 - accuracy: 0.4290
Epoch 3/30
1563/1563 [==============================] - 84s 54ms/step - loss: 1.4472 - accuracy: 0.4824
Epoch 4/30
1563/1563 [==============================] - 130s 83ms/step - loss: 1.2539 - accuracy: 0.5528
Epoch 3/30
1563/1563 [==============================] - 82s 53ms/step - loss: 1.3622 - accuracy: 0.5171
Epoch 5/30
1563/1563 [==============================] - 126s 80ms/step - loss: 1.1471 - accuracy: 0.5944
Epoch 4/30
1563/1563 [==============================] - 82s 5

In [2]:
plot_confusion_matrix(best_model_1, x_test, y_test, 'Matriz de Confusão do Melhor Modelo')

NameError: name 'best_model_1' is not defined